# Signal is proportional to fraction folded

In [18]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../../src')
from util import *
from config import * # where study is defined
import generate_dataset
import plots
import ipynbname

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload



## ∆T –  Does the Tm change with reaction Temp?

### Change in Temp (w/o prediction)
- For each A/C base in ROI, plot mutation fraction vs. T
    - should be linear for some portion, then level off when hydrolysis is faster than DMS reaction


In [19]:
%autoreload 2
for construct in study.df.construct.unique():
    plots.change_in_temp_mut_frac_vs_temperature(study, study.get_samples(), construct)
    savefig2(ipynbname.path(), '[A] Change in temperature (w o prediction)/'+construct)
    if not generate_plots:
        break

No data: ['1_1_S26new_reads' '01_02_S23_reads' 'lauren473_S4' 'lauren471_S2'
 '1_2_S27new_reads' '01_1_S22_reads' '05_2_S25_reads'], 3593-O-flank_1=bi7-ire-DB


### Mutation rate across the family vs pred ∆G (at 37 °C)
- Curve with Predicted ∆G
for each base in ROI, plot mutation fraction vs. pred ∆G (at 37 °C)
- fit each curve to ∆∆Goffset 


In [20]:

%autoreload 2
for sample in study.df['sample'].unique():
    for family in study.get_df(sample=sample).family.unique():
        try:
            plots.mut_rate_across_family_vs_deltaG(study, sample, family)
            savefig2(ipynbname.path(), '[B] Change in temperature (w prediction at 37)/{}/{}'.format(sample, family))
        except:
            print('Error: sample={}, family={}'.format(sample, family))
        if not generate_plots:
            break
    if not generate_plots:
        break

ValueError: 14 columns passed, passed data had 18 columns

### Change in Temp (w/ prediction at T) 
- For each base in ROI at each temp, plot mutation fraction vs. pred ∆G (at that temp) 
    - fit each curve to ∆∆Goffset
    - simple model—the offset should be the same if the algorithm is just off by some amount??


In [ ]:
savefig2(ipynbname.path(), '[C] Change in temperature (w prediction at T)')

<Figure size 640x480 with 0 Axes>

##   ∆t – is the signal in the linear regime at the working reaction time? 
### Change in reaction time
- For each construct at each position, plot Mutation fraction vs. [DMS]
    - scatter plot
    - fit linear region (for now that might just be to the first 3 points 1-5 min)--goodness of fit??




In [ ]:

savefig2(ipynbname.path(), '[D] Change in reaction time')

<Figure size 640x480 with 0 Axes>

## ∆[DMS] -  is the signal in the linear regime at the working concentration?
### Change in [DMS]
- For each construct at each position, plot Mutation fraction vs. [DMS]
    - fit linear region (for now that might just be to the first 3 points 0.5-1% DMS)--goodness of fit??


In [ ]:
savefig2(ipynbname.path(), '[E] Change in [DMS]')

<Figure size 640x480 with 0 Axes>